In [2]:
%cd drive/MyDrive/Crayon/

/content/drive/MyDrive/Crayon


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os 
import zipfile 
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers 
from tensorflow.keras import Model 
import matplotlib.pyplot as plt

In [ ]:
# local_zip = './Dataset/dataset.zip'
# zip_ref = zipfile.ZipFile(local_zip, 'r')
# zip_ref.extractall('./temp')
# zip_ref.close()

In [4]:
base_dir = './temp'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validate')

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'covid_n')

# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'covid_p')

# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'covid_n')

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'covid_p')


In [5]:
# # Add our data-augmentation parameters to ImageDataGenerator
# train_datagen = ImageDataGenerator(rescale = 1./255., rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2,shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

# test_datagen = ImageDataGenerator( rescale = 1.0/255. )

# # Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.)
test_datagen = ImageDataGenerator( rescale = 1.0/255. )



In [16]:
train_generator = train_datagen.flow_from_directory(train_dir, batch_size = 8, class_mode = 'binary', target_size = (150, 150))
validation_generator = test_datagen.flow_from_directory(validation_dir, batch_size = 8, class_mode = 'binary', target_size = (150, 150))

Found 410 images belonging to 2 classes.
Found 50 images belonging to 2 classes.


In [17]:
from tensorflow.keras.applications import VGG16

base_model = VGG16(input_shape=(224, 224,3), include_top=False, weights="imagenet")

58900480/58889256 [==============================] - 1s 0us/step


In [18]:
for layer in base_model.layers:
    layer.trainable = False

In [19]:
from tensorflow.keras.applications import VGG16
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

base_model = Sequential()
base_model.add(VGG16(include_top=False, weights='imagenet', pooling='max'))
base_model.add(Dense(1, activation='sigmoid'))

In [20]:
optimizer = Adam(learning_rate=0.0001)
base_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['acc'])


In [22]:
resnet_history = base_model.fit(train_generator, validation_data = validation_generator, steps_per_epoch = 100, epochs = 10)

Epoch 1/10
  2/100 [..............................] - ETA: 58s - loss: 0.5270 - acc: 0.8750

KeyboardInterrupt: ignored

In [23]:
resnet_history = base_model.fit(train_generator,
                                  epochs=5,
                                  steps_per_epoch=len(train_generator),
                                  validation_data=validation_generator,
                                  validation_steps=len(validation_generator)
                                 ) # name of log files

Epoch 1/5
52/52 [==============================] - 37s 720ms/step - loss: 0.6403 - acc: 0.6683 - val_loss: 0.7479 - val_acc: 0.5000
Epoch 2/5
52/52 [==============================] - 12s 240ms/step - loss: 0.6091 - acc: 0.7073 - val_loss: 0.6682 - val_acc: 0.5600
Epoch 3/5
52/52 [==============================] - 12s 233ms/step - loss: 0.6280 - acc: 0.6854 - val_loss: 0.7199 - val_acc: 0.4400
Epoch 4/5
52/52 [==============================] - 12s 241ms/step - loss: 0.7137 - acc: 0.6341 - val_loss: 0.6941 - val_acc: 0.4800
Epoch 5/5
52/52 [==============================] - 14s 262ms/step - loss: 0.6818 - acc: 0.7049 - val_loss: 0.6893 - val_acc: 0.5600


In [6]:
import os
import tensorflow as tf #tf 2.0.0
import numpy as np

In [7]:
ResNet_model = tf.keras.applications.ResNet152V2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

234561536/234545216 [==============================] - 2s 0us/step


In [8]:
from tensorflow.keras import Model 
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten,GlobalAveragePooling2D
from tensorflow.keras.models import Sequential

# The last 15 layers fine tune
for layer in ResNet_model.layers[:-15]:
    layer.trainable = False

x = ResNet_model.output
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = Dense(units=512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(units=512, activation='relu')(x)
x = Dropout(0.3)(x)
output  = Dense(units=20, activation='softmax')(x)
model = Model(ResNet_model.input, output)


model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 156, 156, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 75, 75, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 pool1_pad (ZeroPadding2D)      (None, 77, 77, 64)   0           ['conv1_conv[0][0]']         

In [9]:
loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss=loss, metrics= ['accuracy'])